# Finding Spark

Whenever we work in Spark the first thing we need is the spark contect (sc).  We are going to use the module `findspark` to get access to the spark context.  First we need to install the module:

In [1]:
! pip install findspark

First we specify the path to spark - which for us is on the local VM:

In [2]:
import findspark
import os
findspark.init('/home/csumb/spark-1.6.0-bin-hadoop2.6')

Now we can import pyspark and get the spark context:

In [3]:
import pyspark
try: 
    print(sc)
except NameError:
    sc = pyspark.SparkContext()
    print(sc)

# Creating an RDD

From the Spark documentation:

_"A Resilient Distributed Dataset (RDD), the basic abstraction in Spark, represents an immutable, partitioned collection of elements that can be operated on in parallel."_

_"Parallelized collections are created by calling SparkContext’s parallelize method on an existing iterable or collection in your driver program. The elements of the collection are copied to form a distributed dataset that can be operated on in parallel."_ 

For example, here is how to create a parallelized collection holding the numbers 1 to 5:


In [13]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)

The RDD will now execute operations in parallel, for example to add up elements of list:

In [18]:
distData.reduce(lambda a, b: a + b)

15

### External Data Sources

We can also create RDDs from external data sources such as Hadoop, Amazon S3 and files. Here we will create a text file RDD.  NOte that we must use absolute paths since this code is pushed onto the Spark cluster - it is not run in the context of this notebook:

In [29]:
rdd = sc.textFile('/home/csumb/data-science-for-search/data/bike-item-titles.txt')
print(rdd)

MapPartitionsRDD[19] at textFile at NativeMethodAccessorImpl.java:-2


### Basics - transformations and actions

The RDD is not loaded in memory - it is just a pointer to the file.  Spark allows us to apply transformations to the RDD - but these are computed immediately - Spark is intentionally lazy.  Nothing is computed until we execute an action, at which point the Spark driver creates tasks to run on separate nodes in the Spark cluster.  Each node executes the transformations and actions and returns the results to the driver.   

http://vishnuviswanath.com/spark_rdd.html

### Counting Words

To illustrate RDD basics, consider the simple program below which counts the number of words in the text file rdd we created earlier:

In [33]:
words_per_line = rdd.map(lambda s: len(s.split()))
# print(words_per_line)
total_words = words_per_line.reduce(lambda a, b: a + b)
# print(total_words)

PythonRDD[22] at RDD at PythonRDD.scala:43
106983


In [38]:
print(words_per_line.take(10))

[13, 13, 14, 10, 6, 8, 16, 12, 11, 14]


To reiterate - `words_per_line` is applies a transformation to the rdd - it is lazy and not evaluated until we apply an action - such as `reduce()`.  We can inspect the transformations applied to the RDD using the `toDebugString()` method:

In [40]:
print(words_per_line.toDebugString())

(1) PythonRDD[22] at RDD at PythonRDD.scala:43 [Memory Serialized 1x Replicated]
 |       CachedPartitions: 1; MemorySize: 10.0 KB; ExternalBlockStoreSize: 0.0 B; DiskSize: 0.0 B
 |  MapPartitionsRDD[19] at textFile at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]
 |  /home/csumb/data-science-for-search/data/bike-item-titles.txt HadoopRDD[18] at textFile at NativeMethodAccessorImpl.java:-2 [Memory Serialized 1x Replicated]
